In [1]:
import os
import pickle
import pandas as pd

wd2 = "/Users/tmm/Documents/GitHub/STA160-Project/STA160-Jobs-Market-Project/Working/data"
os.chdir(wd2)

jobs_scrub_in = open('raw_jobs_data.pickle', 'rb')
jobs_scrub = pickle.load(jobs_scrub_in)


jobs_scrub.columns = ['Search','Title','Description', 'Skills', 'Location', 'Latitude', 'Longitude', 'Salary', 'URL']

wd3 = "/Users/tmm/Documents/GitHub/STA160-Project/STA160-Jobs-Market-Project/Working/data_wrangling"
os.chdir(wd3)

In [2]:
def skills_clean(jobs_scrub, field):
    """
    (Purpose)
    Cleans up the skills for each listing. The raw data has 'Preferred Skills', '\r', and \n'
    that need to be removed in order to have a cleaner format. 
    
    (Arguments)
    field : Skills
    
    (Returns)
    skills : list of scrubbed skills
    
    """
    nrows, ncols = jobs_scrub.shape
    
    removals1 = ['Preferred Skills', '\r', '\n']
    for i in range(nrows):
        for j in removals1:
            holder1 = jobs_scrub[field][i].replace(j,'')
            jobs_scrub.loc[i,field] = holder1
        #jobs_scrub.loc[i,field] = jobs_scrub.loc[i,field].split('  ')
    
    skills_split = lambda jobs_scrub : jobs_scrub[field].split('  ')
    jobs_scrub[field] = jobs_scrub.apply(skills_split, axis = 1)
    
    none_removal = lambda jobs_scrub : list(filter(None, jobs_scrub[field]))
    jobs_scrub[field] = jobs_scrub.apply(none_removal, axis = 1)
    
    jobs_scrub[field] = jobs_scrub[field].apply(lambda jobs_scrub: ', '.join(map(str, jobs_scrub)))
    
    return jobs_scrub[field]

In [3]:
def description_scrub(jobs_scrub, field):
    """
    (Purpose)
    Cleans up the description for each listing. The raw data has '*', '\r', \n', and 
    'Applicants must be authorized to work in the U.S.' that need to be removed in order 
    to have a cleaner format. 
    
    (Arguments)
    field : Description
        
    (Returns)
    description : list of scrubbed descriptions
    
    """
    nrows, ncols = jobs_scrub.shape
    
    removals = ['*','\r','\n','Applicants must be authorized to work in the U.S.']
    for i in range(nrows):
        for j in removals:
            holder = jobs_scrub[field][i].replace(j,'')
            jobs_scrub.loc[i,field] = holder
    return jobs_scrub[field]

In [4]:
def location_clean(jobs_scrub, field):
    """
    (Purpose)
    Seperates the locations for each listing into city and states, which is a cleaner format for
    conducting analysis later on. 
    
    (Arguments)
    field : Location
    
    (Return)
    city : list of cities for job listings
    state : list of state for job listings
    
    """
    nrows, ncols = jobs_scrub.shape
    cities = []
    states = []
    for i in range(nrows):
        location_split = jobs_scrub[field][i].split(', ')
        cities.append(location_split[0])
        states.append(location_split[1])
    return cities, states

In [5]:
def salary_clean(jobs_scrub, field):
    """
    (Purpose)
    Cleans up the salary portion for each listing if there is data present. Seperates the salary
    estimates into a minimum and maximum salary estimate. Also, the format is converted into thousands
    and numeric. 
    
    (Arguments)
    field : Salary
    
    (Return)
    min_salary : list of minimum salary estimates
    max_salary : list of maximum salary estimates
    
    """
    nrows, ncols = jobs_scrub.shape
    min_salary = []
    max_salary = []
    mean_salary = []

        
    for i in range(nrows):
        for j in range(len(jobs_scrub[field][i])):
            if len(jobs_scrub[field][i][j]) >= 2:
                jobs_scrub[field][i][j] = jobs_scrub[field][i][j] + '000'
                jobs_scrub[field][i][j] = int(jobs_scrub[field][i][j])
    
    for i in range(nrows):
        if len(jobs_scrub[field][i]) == 0:
            jobs_scrub[field][i].append(0)
            jobs_scrub[field][i].append(0)
    
    for i in range(nrows):
        min_salary.append(jobs_scrub[field][i][0])
        max_salary.append(jobs_scrub[field][i][1])
        mean_salary.append((jobs_scrub[field][i][0] + jobs_scrub[field][i][1]) / 2)
        
    for i in range(nrows):
        if min_salary[i] == 0:
            min_salary[i] = 'Unknown'
        if max_salary[i] == 0:
            max_salary[i] = 'Unknown'
        if mean_salary[i] ==0:
            mean_salary[i] = 'Unknown'
            
            
    return min_salary, max_salary, mean_salary


In [6]:
description_list = description_scrub(jobs_scrub, 'Description')
skills_list = skills_clean(jobs_scrub, 'Skills')
locations_list = location_clean(jobs_scrub, 'Location')
wage_list = salary_clean(jobs_scrub, 'Salary')

jobs_scrub['Description'] = description_list
jobs_scrub['Skills'] = skills_list
jobs_scrub['Cities'] = locations_list[0]
jobs_scrub['States'] = locations_list[1]
jobs_scrub['Min_Salary'] = wage_list[0]
jobs_scrub['Max_Salary'] = wage_list[1]
jobs_scrub['Mean_Salary'] = wage_list[2]
jobs_scrub = jobs_scrub[jobs_scrub['Description'] != '']

jobs_scrub = jobs_scrub[['Search', 
                         'Title',
                         'Description',
                         'Skills',
                         'Location',
                         'Cities', 
                         'States', 
                         'Latitude', 
                         'Longitude', 
                         'Min_Salary', 
                         'Max_Salary', 
                         'Mean_Salary', 
                         'URL']]

In [7]:
jobs_scrub

,Search,Title,Description,Skills,Location,Cities,States,Latitude,Longitude,Min_Salary,Max_Salary,Mean_Salary,URL
0,Data Scientist,Data Scientist,Are you a Data Scientist who has machine learn...,"Statistics, Machine Learning, Applied Mathematics","Los Angeles, CA",Los Angeles,CA,34.052234,-118.243685,100000,200000,150000,https://www.cybercoders.com/data-scientist-job...
1,Data Scientist,Data Scientist,If you are a Data Scientist with 4+ years data...,"Data Science, Machine Learning, R, Python, SQL...","San Francisco, CA",San Francisco,CA,37.774929,-122.419415,110000,175000,142500,https://www.cybercoders.com/data-scientist-job...
2,Data Scientist,Data Scientist,"If you are a Data Scientist with experience, p...","Statistics, Mathematics, Computer Science, Mac...","Woodland Hills, CA",Woodland Hills,CA,34.165357,-118.608975,Unknown,Unknown,Unknown,https://www.cybercoders.com/data-scientist-job...
3,Data Scientist,Data Scientist,If you are a Data Scientist who enjoys buildin...,"Python, Java, SQL (BigQuery/Oracle and Hive), ...","Los Angeles, CA",Los Angeles,CA,34.052234,-118.243685,120000,160000,140000,https://www.cybercoders.com/data-scientist-job...
4,Data Scientist,Data Scientist,"If you are a Data Scientist with experience, p...","R Language, Python, Datasets, SQL Queries","Austin, TX",Austin,TX,30.267153,-97.743061,Unknown,Unknown,Unknown,https://www.cybercoders.com/data-scientist-job...
5,Data Scientist,Data Scientist,If you are a Data Scientist who has Python and...,"Full Stack, Cassandra, ElasticSearch, Solr, Da...","Cambridge, MA",Cambridge,MA,42.373616,-71.109733,110000,150000,130000,https://www.cybercoders.com/data-scientist-job...
6,Data Scientist,Data Scientist,"If you are a Data Scientist with experience, p...","Machine Learning, Tensorflow, Statistical/Mach...","Bethesda, MD",Bethesda,MD,38.984652,-77.094709,100000,120000,110000,https://www.cybercoders.com/data-scientist-job...
7,Data Scientist,Data Scientist,If you are a Data Scientist who enjoys develop...,"Python, Java, SQL (BigQuery/Oracle and Hive), ...","Los Angeles, CA",Los Angeles,CA,34.052234,-118.243685,120000,160000,140000,https://www.cybercoders.com/data-scientist-job...
8,Data Scientist,Data Scientist,"Data Scientist - data mining, data sets, stati...","Data Mining, data sets, Statistical Modeling, ...","Plymouth Meeting, PA",Plymouth Meeting,PA,40.102332,-75.274347,120000,160000,140000,https://www.cybercoders.com/data-scientist-job...
9,Data Scientist,Data Scientist,"Data Scientist - Machine Learning, Python, Tab...","Machine Learning, Python, Tableau, Hadoop, R, ...","Trenton, NJ",Trenton,NJ,40.220582,-74.759717,130000,150000,140000,https://www.cybercoders.com/data-scientist-job...


In [8]:
# copy cleaned up jobs_data
master_data = jobs_scrub.copy()

# dataframe for data scientist
data_scientist_df = master_data.copy()
data_scientist_df = data_scientist_df[data_scientist_df['Search'].str.contains('Data Scientist')]
data_scientist_df = data_scientist_df.reset_index(drop = True)

# dataframe for data engineer
data_engineer_df = master_data.copy()
data_engineer_df = data_engineer_df[data_engineer_df['Search'].str.contains('Data Engineer')]
data_engineer_df = data_engineer_df.reset_index(drop = True)

# dataframe for data analyst
data_analyst_df = master_data.copy()
data_analyst_df = data_analyst_df[data_analyst_df['Search'].str.contains('Data Analyst')]
data_analyst_df = data_analyst_df.reset_index(drop = True)

# dataframe for business intelligence
business_intelligence_df = master_data.copy()
business_intelligence_df = business_intelligence_df[business_intelligence_df['Search'].str.contains('Business Intelligence')]
business_intelligence_df = business_intelligence_df.reset_index(drop = True)

In [10]:
w4 = '/Users/tmm/Documents/GitHub/STA160-Project/STA160-Jobs-Market-Project/Working/data'
os.chdir(w4)

master_data.to_csv('jobs_data.csv',index = False)
data_scientist_df.to_csv('data_scientist_data.csv', index = False)
data_engineer_df.to_csv('data_engineer_data.csv', index = False)
data_analyst_df.to_csv('data_analyst_data.csv', index = False)
business_intelligence_df.to_csv('BI_data.csv', index = False)

master_data_out = open('jobs_data.pickle', 'wb')
pickle.dump(master_data, master_data_out)
master_data_out.close()

data_scientist_out = open('data_scientist_data.pickle', 'wb')
pickle.dump(data_scientist_df, data_scientist_out)
data_scientist_out.close()

data_engineer_out = open('data_engineer_data.pickle', 'wb')
pickle.dump(data_engineer_df, data_engineer_out)
data_engineer_out.close()

data_analyst_out = open('data_analyst_data.pickle', 'wb')
pickle.dump(data_analyst_df, data_analyst_out)
data_analyst_out.close()

BI_out = open('BI_data.pickle', 'wb')
pickle.dump(business_intelligence_df, BI_out)
BI_out.close()